In [1]:
import os
import re
import json
import fitz  # PyMuPDF
import unicodedata
import time
import pandas as pd
from pathlib import Path
from typing import Dict, List, Any

# ==============================================================================
# CONFIGURAÇÕES
# ==============================================================================
# Ajuste estes caminhos conforme seu ambiente
INPUT_DIR = Path('data/pdf/categorized/simplified')
OUTPUT_DIR = Path('data/json/llm_ready')

# ==============================================================================
# FUNÇÕES DE LIMPEZA SUAVE (LLM FRIENDLY)
# ==============================================================================

def limpeza_llm_friendly(texto: str) -> str:
    """
    Limpeza conservadora focada em manter a estrutura semântica para LLMs.
    Preserva: Acentos, pontuação, preposições, case (caixa alta/baixa).
    Remove: Quebras de linha indesejadas, cabeçalhos repetitivos e hifenização de fim de linha.
    """
    if not texto:
        return ""

    # 1. Normalização Unicode (NFC é melhor para compatibilidade visual e LLMs)
    texto = unicodedata.normalize("NFC", texto)

    # 2. Corrigir hifenização de fim de linha (ex: "aero-\nnave" -> "aeronave")
    # Procura letra + hífen + newline + espaços + letra e junta
    texto = re.sub(r'(\w)-\n\s*(\w)', r'\1\2', texto)

    linhas = texto.split('\n')
    linhas_limpas = []

    for linha in linhas:
        linha = linha.strip()
        if not linha:
            continue

        # 3. Remover lixo comum de rodapé/cabeçalho de PDF
        # Remove contadores de página (ex: "1 de 5", "01/05", "3 of 10")
        if re.match(r'^\d+\s+(?:de|of|/)\s+\d+$', linha, re.IGNORECASE):
            continue
        # Remove linhas que são apenas datas soltas ou códigos CENIPA isolados
        if re.match(r'^.*CENIPA.*\d{4}$', linha, re.IGNORECASE):
            continue

        linhas_limpas.append(linha)

    # 4. Unir linhas
    # Para LLMs, é melhor ter um fluxo de texto contínuo do que quebras de linha artificiais do PDF.
    # Juntamos com espaço. (Opcional: Tentar detectar parágrafos reais, mas espaço é mais seguro)
    texto_fluido = " ".join(linhas_limpas)

    # 5. Normalizar espaços múltiplos (ex: tabulações ou espaços duplos viram um só)
    texto_fluido = re.sub(r'\s+', ' ', texto_fluido)

    return texto_fluido.strip()

def ignorar_cabecalho_advertencia(texto_completo: str) -> str:
    """
    Pula o aviso legal inicial. Ajustado para ser flexível com acentos e case.
    """
    # Marcadores flexíveis (case insensitive e acento insensitive)
    # (?i) ativa case insensitive
    marcadores_conteudo = [
        r'(?i)\d+\.?\s+informa[cç][oõ]es\s+factuais', # Informações Factuais
        r'(?i)\d+\.?\s+hist[oó]rico\s+(?:do\s+)?voo', # Histórico (do) Voo
        r'(?i)\d+\.?\d*\.?\s+hist[oó]rico\s+(?:do\s+)?voo'
    ]

    posicao_inicio = -1

    for marcador in marcadores_conteudo:
        match = re.search(marcador, texto_completo)
        if match:
            if posicao_inicio == -1 or match.start() < posicao_inicio:
                posicao_inicio = match.start()

    if posicao_inicio != -1:
        return texto_completo[posicao_inicio:]
    else:
        # Se não achar, tenta retornar tudo, mas loga aviso
        # print("  ⚠️ Aviso: Início exato não encontrado, usando texto completo.")
        return texto_completo

# ==============================================================================
# FUNÇÕES DE EXTRAÇÃO DE SEÇÃO (COM REGEX FLEXÍVEL)
# ==============================================================================

# Regex compilados para performance e reuso.
# Nota: Usamos (?i) para ignorar case e [aá] para aceitar com ou sem acento.
MARCADORES_FIM_UNIVERSAIS = [
    r'(?i)\d+\.?\s+hist[oó]rico\s+(?:do\s+)?voo',
    r'(?i)\d+\.?\s+an[aá]lise',
    r'(?i)\d+\.?\s+coment[aá]rios', # Alguns relatórios usam Comentários em vez de Análise
    r'(?i)\d+\.?\s+fatos', # Fatos pode aparecer
    r'(?i)\d+\.?\s+conclus[oõ]es',
    r'(?i)\d+\.?\s+fatores\s+contribuintes',
    r'(?i)\d+\.?\s+recomenda[cç][oõ]es',
    r'(?i)\d+\.?\s+a[cç][oõ]es\s+corretivas',
    r'(?i)ap[êe]ndice',
    r'(?i)anexos'
]

def encontrar_secao(texto_completo: str, marcadores_inicio: List[str], marcadores_fim: List[str]) -> str:
    """
    Extrai trecho entre um marcador de início e o próximo marcador de fim encontrado.
    """
    inicio_secao = -1
    fim_marcador_inicio = -1

    # Tenta encontrar o início
    for marcador in marcadores_inicio:
        match = re.search(marcador, texto_completo)
        if match:
            inicio_secao = match.start()
            fim_marcador_inicio = match.end()
            break

    if inicio_secao == -1:
        return ""

    # Procura o fim da seção (o marcador de fim mais próximo APÓS o início)
    fim_secao = len(texto_completo)
    posicao_minima_fim = float('inf')

    for marcador in marcadores_fim:
        # Busca marcadores apenas no texto após o cabeçalho da seção atual
        # Isso evita achar um marcador que está lá atrás
        matches = list(re.finditer(marcador, texto_completo[fim_marcador_inicio:]))
        if matches:
            # Pega o primeiro match que aparecer depois do nosso início
            primeiro_match = matches[0]
            posicao_absoluta = primeiro_match.start() + fim_marcador_inicio

            # Se achou um marcador de fim válido e ele está antes do que já tínhamos achado
            if posicao_absoluta < posicao_minima_fim:
                posicao_minima_fim = posicao_absoluta

    if posicao_minima_fim != float('inf'):
        fim_secao = posicao_minima_fim

    return texto_completo[fim_marcador_inicio:fim_secao].strip()

def extrair_historico_voo(texto: str) -> str:
    marcadores = [r'(?i)\d+\.?\s+hist[oó]rico\s+(?:do\s+)?voo']
    return encontrar_secao(texto, marcadores, MARCADORES_FIM_UNIVERSAIS)

def extrair_analise(texto: str) -> str:
    # Pega tanto "Análise" quanto "Comentários" (comum em sumas antigas)
    marcadores = [r'(?i)\d+\.?\s+an[aá]lise', r'(?i)\d+\.?\s+coment[aá]rios']
    return encontrar_secao(texto, marcadores, MARCADORES_FIM_UNIVERSAIS)

def extrair_fatores_contribuintes_texto(texto: str) -> str:
    marcadores = [r'(?i)\d+\.?\s+fatores\s+contribuintes']
    return encontrar_secao(texto, marcadores, MARCADORES_FIM_UNIVERSAIS)

# ==============================================================================
# PROCESSAMENTO DE FATORES (RETORNA LISTA ESTRUTURADA)
# ==============================================================================

def processar_lista_fatores(texto_fatores: str) -> List[str]:
    """
    Processa o texto bruto da seção de fatores para extrair uma lista limpa.
    Mantém a lógica original mas adaptada para lidar com acentos.
    """
    if not texto_fatores:
        return []

    # Aqui podemos quebrar por ' - ' ou quebras de linha, dependendo do formato
    # Como normalizamos tudo para espaço no clean_llm, talvez tenhamos perdido os \n.
    # SE a seção de fatores for uma lista, ela geralmente tem estrutura.
    # TRUQUE: Se o texto_fatores vier do 'limpeza_llm_friendly', ele não tem \n.
    # Para fatores, talvez seja melhor processar ANTES de remover todos os \n ou
    # usar heurísticas.

    # Opção A: Tentar separar por padrões comuns de fatores contribuintes
    # Fatores geralmente terminam com "CONTRIBUIU" ou "INDETERMINADO".

    fatores_encontrados = []

    # Regex para capturar "Nome do Fator - Classificação"
    # Ex: "Planejamento de voo - Contribuiu"
    # Aceita acentos em "Não contribuiu"
    padrao_fator = r'(?i)(.*?)\s+(-|–)\s*(contribuiu|indeterminado|n[aã]o\s+contribuiu)'

    matches = re.finditer(padrao_fator, texto_fatores)

    for match in matches:
        nome_fator = match.group(1).strip()
        classificacao = match.group(3).strip()

        # Limpa sujeira do início do nome do fator se houver (ex: bullets)
        nome_fator = re.sub(r'^[\•\-\*\d\.]+\s*', '', nome_fator)

        if len(nome_fator) > 3:
            fatores_encontrados.append(f"{nome_fator} ({classificacao})")

    # Fallback: Se não achou estrutura padrão, retorna o texto bruto para o LLM decidir
    if not fatores_encontrados and len(texto_fatores) > 10:
        return [texto_fatores]

    return fatores_encontrados

# ==============================================================================
# PIPELINE PRINCIPAL
# ==============================================================================

def extrair_texto_pdf(pdf_path: str) -> str:
    try:
        with fitz.open(pdf_path) as doc:
            text = ""
            for page in doc:
                text += page.get_text() + "\n"
        return text
    except Exception as e:
        print(f"Erro ao ler {pdf_path}: {e}")
        return ""

def processar_relatorio(pdf_path: Path) -> Dict[str, Any]:
    nome_arquivo = pdf_path.name
    print(f"🔄 Processando: {nome_arquivo}")

    # 1. Extração Bruta
    texto_bruto = extrair_texto_pdf(str(pdf_path))
    if not texto_bruto:
        return None

    # 2. Limpeza Suave (Preserva Preposições e Acentos)
    texto_limpo = limpeza_llm_friendly(texto_bruto)

    # 3. Remover Advertência Inicial
    texto_conteudo = ignorar_cabecalho_advertencia(texto_limpo)

    # 4. Extração de Seções
    historico = extrair_historico_voo(texto_conteudo)
    analise = extrair_analise(texto_conteudo)

    # Extrai texto bruto dos fatores e depois processa
    fatores_texto_bruto = extrair_fatores_contribuintes_texto(texto_conteudo)
    fatores_lista = processar_lista_fatores(fatores_texto_bruto)

    # Validação simples
    if not any([historico, analise, fatores_lista]):
        print(f"  ⚠️  Aviso: Nenhuma seção relevante detectada em {nome_arquivo}")
        # Retorna o objeto mesmo assim, pode ser útil ver o que falhou

    return {
        "metadata": {
            "arquivo": nome_arquivo,
            "processado_em": time.strftime("%Y-%m-%d %H:%M:%S")
        },
        "conteudo": {
            "historico_voo": historico,
            "analise": analise,
            "fatores_contribuintes": fatores_lista
        },
        "texto_completo_limpo": texto_conteudo # Opcional: útil para Debug ou RAG
    }

def main():
    if not INPUT_DIR.exists():
        print(f"Diretório de entrada não encontrado: {INPUT_DIR}")
        return

    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    arquivos = list(INPUT_DIR.glob("*.pdf"))

    print(f"Encontrados {len(arquivos)} arquivos.")

    sucessos = 0
    for i, arquivo in enumerate(arquivos):
        try:
            resultado = processar_relatorio(arquivo)
            if resultado:
                nome_json = arquivo.stem + ".json"
                with open(OUTPUT_DIR / nome_json, 'w', encoding='utf-8') as f:
                    json.dump(resultado, f, ensure_ascii=False, indent=4)
                sucessos += 1
        except Exception as e:
            print(f"❌ Erro crítico em {arquivo.name}: {e}")

    print(f"\nProcessamento concluído. {sucessos}/{len(arquivos)} processados com sucesso.")

if __name__ == "__main__":
    main()

Encontrados 1567 arquivos.
🔄 Processando: A-076CENIPA2018_PUJYZ_atualizadoRFS_MTA.pdf
🔄 Processando: A-135CENIPA2019_PRZJO_15ABR2021.pdf
🔄 Processando: BRXXX2_28_07_2024_PUB..pdf
🔄 Processando: CJB_encdivulgacao.pdf
🔄 Processando: COBT_encdivulgacao.pdf
🔄 Processando: CP-2609_22_03_2016-AC.pdf
🔄 Processando: CP2615_18_04_2016-ACID..pdf
🔄 Processando: N114U_02_02_2013_-_ACID.pdf
🔄 Processando: N2015S_26DEZ2009_AC..pdf
🔄 Processando: N209B_12_03_2013_-_ACID.pdf
🔄 Processando: n6523n_09_04_13.pdf
🔄 Processando: N8540A_07_07_09_AC_1.pdf
🔄 Processando: PP-AFK_14_06_2024-PUB..pdf
🔄 Processando: PP-AMM_18_03_2011-AC.pdf
🔄 Processando: PP-ART_28_01_2019_AC..pdf
🔄 Processando: PP-ARX_27_06_2018_AC..pdf
🔄 Processando: PP-BAO_25_10_2015-AC.pdf
🔄 Processando: PP-BCC_16_12_2014_AC..pdf
🔄 Processando: PP-BOZ_13_10_2021-AC.VER.PUB..pdf
🔄 Processando: PP-CIE_24_03_2012_IG.pdf
🔄 Processando: PP-CLD_05_01_2021-AC..pdf
🔄 Processando: PP-CTD_23_01_2016.pdf
🔄 Processando: PP-DFM_20_08_2019-AC..pdf
🔄 Proces